<a href="https://colab.research.google.com/github/migostro/laboratorio-de-visao/blob/main/3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-image==0.19.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.5 MB 5.6 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np
from google.colab import files, drive
import os
from skimage import io, color
from skimage import segmentation, measure

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

/usr/local/lib/python3.7/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


# Helper Functions

In [ ]:
# list_filepaths(): list filepath for every filepath on a root folder
# pre-condition: (root path, empty list)
# post-condition: list with every filepath on root path
def list_filepaths(path, file_extension, filepaths = []):
    if os.path.isdir(path):
        for filename in os.listdir(path):
            filepath = os.path.join(path, filename)
            if os.path.isfile(filepath) and file_extension in filename: filepaths.append(filepath) # Adiciona apenas caminhos que são de arquivos (que no nosso caso são imagens)
            else: list_filepaths(filepath, file_extension, filepaths)

    return filepaths

# Pipeline

In [ ]:
# 0 FILE READING

drive.mount('/content/drive')
#root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new'
root = '/content/drive/MyDrive/ime/mac0417/ep'
originalPath = root + "/originalDataSet_resized"
segmented_path = root + "/segmented_resized"
groundTruthDataPath = root + "/groundTruth_resized"

Mounted at /content/drive


In [ ]:
def create_list_segmented(original_paths):
    new_paths = []
    for path in original_paths:
        new_path = path.split('/')
        new_path[-6] = 'segmented_resized'
        new_format_photo = new_path[-1].split('.')
        new_format_photo[-1] = 'png'

        new_path[-1] = new_format_photo[0] + '.' + 'png'

        aux_new_path = '/'
        for i in new_path:
            aux_new_path = os.path.join(aux_new_path, i)
        new_paths.append(aux_new_path)
    return new_paths


In [ ]:
originalPaths = list_filepaths(originalPath, 'JPG', [])
segmentedPaths = create_list_segmented(originalPaths)

print(f'# of images @ {originalPath}: {len(originalPaths)}')
print(f'# of images @ {segmented_path}: {len(segmentedPaths)}')
print(originalPaths)
print(segmentedPaths)

# of images @ /content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized: 1080
# of images @ /content/drive/MyDrive/ime/mac0417/ep/segmented_resized: 1080
['/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0856.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0857.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0858.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0859.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0860.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0861.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0862.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/bo

In [ ]:
#print(len(segmentedPaths))
#print(len(filteredOriginalPaths))

In [ ]:
classes = ['borrachas', 'cartas', 'celulares', 'conchas', 'copos', 'dados', 'estatuas', 'lapis', 'tubos', 'vasos']
num_samples = len(originalPaths)
num_features = 11
num_classes = len(classes)
X = np.zeros([num_samples, num_features])
Y = np.zeros([num_samples, num_classes])

# https://scikit-learn.org/stable/modules/feature_extraction.html
for i in range(num_samples):
    for j in range(num_classes):
        if originalPaths[i].split('/')[-3] == classes[j]:
            Y[i, j] = 1

In [ ]:
# create feature vector
for i in range(num_samples):
    print(f'[{i}]')
    img_original = io.imread(originalPaths[i])

    #if img_segmented
    #print(segmentedPaths[i])
    img_segmented = io.imread(segmentedPaths[i])
    #print(img_segmented.shape)
    cut = np.multiply(img_original, img_segmented)

    label_img = measure.label(cut)
    regions = measure.regionprops(label_img)

    if(len(regions) > 1): # isolate_largest_area
        largest_region = max(regions, key=lambda region: region.area)

    if len(regions) == 1:
        largest_region = regions[0]

    X[i, 0] = np.average(cut)
    X[i, 1] = largest_region.area
    X[i, 2] = largest_region.solidity
    X[i, 3] = largest_region.eccentricity
    X[i, 4] = np.amax(cut)
    X[i, 5] = largest_region.feret_diameter_max
    X[i, 6] = largest_region.perimeter
    X[i, 7] = largest_region.orientation
    X[i, 8] = largest_region.extent
    X[i, 9] = largest_region.axis_major_length
    X[i, 10] = largest_region.axis_minor_length

In [ ]:
print(X, Y)
print(X.shape, Y.shape)

[[7.98945538e+00 4.30000000e+02 5.44303797e-01 ... 4.47916667e-01
  3.69979521e+01 2.34702486e+01]
 [7.98052588e+00 3.29000000e+02 5.10077519e-01 ... 3.70495495e-01
  4.14701089e+01 2.08027013e+01]
 [8.08020066e+00 3.37000000e+02 5.11380880e-01 ... 3.82086168e-01
  5.00870109e+01 1.65844915e+01]
 ...
 [2.88914507e+01 1.03000000e+02 6.51898734e-01 ... 4.08730159e-01
  1.84224983e+01 1.00725803e+01]
 [3.24614960e+01 1.30000000e+02 5.01930502e-01 ... 3.38541667e-01
  2.89869091e+01 1.18986072e+01]
 [3.89178091e+01 1.80000000e+02 4.14746544e-01 ... 3.02521008e-01
  3.69688495e+01 1.43340213e+01]] [[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1080, 11) (1080, 10)


In [ ]:
alphas = np.arange(0.1, 1.1, 0.1)
epsilons = np.array([1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8])
layers = np.arange(10, 101, 10)

print(alphas)
print(epsilons)
print(layers)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[1.e-03 1.e-04 1.e-05 1.e-06 1.e-07 1.e-08]
[ 10  20  30  40  50  60  70  80  90 100]


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

alphas = np.arange(0.1, 1.1, 0.1)
epsilons = np.array([1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8])
layers = np.arange(10, 101, 10)

#cases = np.zeros([len(alphas), len(epsilons), len(layers), 4])
cases = []
for a in range(len(alphas)):
    for e in range(len(epsilons)):
        for l in range(len(layers)):
            
            # create the classifier
            classifier = MLPClassifier(solver='sgd', alpha=alphas[a], epsilon=epsilons[e], hidden_layer_sizes=(layers[l]), max_iter=3000, n_iter_no_change=200, random_state=1)

            # USANDO APENAS X E Y ENQUANTO NAO FUNCIONA
            classifier.fit(X_train, Y_train)

            score = classifier.score(X_train, Y_train)

            #cases[a, e, l, 0] = score
            #cases[a, e, l, 1] = alphas[a]
            #cases[a, e, l, 2] = epsilons[e]
            #cases[a, e, l, 3] = layers[l]

            vetor = []

            vetor.append(alphas[a])
            vetor.append(epsilons[e])
            vetor.append(layers[l])
            vetor.append(score)

            cases.append(vetor)
    print(a)
            


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_m

0


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_m

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_m

2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_m

3


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_m

4
5
6
7
8
9


In [ ]:
print(np.max(cases[:0]))

In [ ]:
print(cases)

[[0.1, 0.001, 10, 0.0], [0.1, 0.001, 20, 0.0], [0.1, 0.001, 30, 0.022222222222222223], [0.1, 0.001, 40, 0.003703703703703704], [0.1, 0.001, 50, 0.009876543209876543], [0.1, 0.001, 60, 0.03209876543209877], [0.1, 0.001, 70, 0.043209876543209874], [0.1, 0.001, 80, 0.02962962962962963], [0.1, 0.001, 90, 0.0], [0.1, 0.001, 100, 0.011111111111111112], [0.1, 0.0001, 10, 0.0], [0.1, 0.0001, 20, 0.0], [0.1, 0.0001, 30, 0.022222222222222223], [0.1, 0.0001, 40, 0.003703703703703704], [0.1, 0.0001, 50, 0.009876543209876543], [0.1, 0.0001, 60, 0.03209876543209877], [0.1, 0.0001, 70, 0.043209876543209874], [0.1, 0.0001, 80, 0.02962962962962963], [0.1, 0.0001, 90, 0.0], [0.1, 0.0001, 100, 0.011111111111111112], [0.1, 1e-05, 10, 0.0], [0.1, 1e-05, 20, 0.0], [0.1, 1e-05, 30, 0.022222222222222223], [0.1, 1e-05, 40, 0.003703703703703704], [0.1, 1e-05, 50, 0.009876543209876543], [0.1, 1e-05, 60, 0.03209876543209877], [0.1, 1e-05, 70, 0.043209876543209874], [0.1, 1e-05, 80, 0.02962962962962963], [0.1, 1e-

In [ ]:
maior = 0
for i in range(len(cases)):
    if cases[i][3] > cases[maior][3]:
        maior = i
print(cases[maior,:])

TypeError: ignored

In [ ]:
np.sum(Y)

In [ ]:
classifier.predict(X[4:10,:])

In [ ]:
# calculate the avg of the right predicts for the test set
classifier.score(X_train, Y_train)

In [ ]:
print(classifier.score(X_train, Y_train))
print(classifier.predict(X_train))
print(Y_train)

In [ ]:
alphas = np.arange(0.01, 1, 0.01)
epsilons = np.array([1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8])
layers = np.arange(10, 100, 10)

cases = np.zeros([len(alphas), len(epsilons), len(layers), 4])

print(cases[:,:,:,0])

In [ ]:
# https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html#sphx-glr-auto-examples-neural-networks-plot-mlp-alpha-py
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

ax = plt.subplot()
cm_bright = ListedColormap(["#FF0000", "#00FF00", "#0000FF","#F0000F","#0F00F0","#FFFFFF","#F0F0F0","#0F0F0F","#0FFFF0","#000000",])
ax.scatter(
    X[:, 0],
    X[:, 1],
    c=Y,
    cmap=cm_bright,
    edgecolors="black",
    s=25,
)
plt.show()